<a href="https://colab.research.google.com/github/lweislo/UCI_data/blob/main/Modeling_Team_Ranking_points.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import files
import numpy as np

In [ ]:
#UPLOAD THE POINTS MODEL WIDE DATA FORM
uploaded = files.upload()
for fn in uploaded.keys():
    print('User uploaded file "{name}"'.format(
        name=fn, length=len(uploaded[fn])))
for item in uploaded:
    ptm = pd.read_csv(item, header=0)

In [ ]:
#Check thecolumns
vv = ptm.columns
vv

In [ ]:
#Change the wide data to long
ptm = pd.melt(ptm, id_vars='Position', value_vars=vv)
#Get rid of nulls
ptm = ptm.dropna()

In [ ]:
#Make a new long dataframe
dfp = pd.DataFrame()
dfp['Rank'] = ptm.Position
dfp['Code'] = ptm.variable
dfp['Points'] = ptm.value

In [ ]:
#UPLOAD 2020-2022 CODED RESULTS FILE
uploaded = files.upload()
for fn in uploaded.keys():
    print('User uploaded file "{name}"'.format(
        name=fn, length=len(uploaded[fn])))
for item in uploaded:
    res = pd.read_csv(item, header=0)
#Check the length
len(res)

In [ ]:
#MERGE RESULTS AND LONG MODEL
pmod = res.merge(dfp, on=['Rank','Code'], how="left")
#Check the length hasn't changed
len(pmod)

In [ ]:
#To visualize, pivot with values the recalculated point totals
rda = pmod.groupby(['Team','Type'])['Points_y'].sum().reset_index()
rda.pivot(index='Team', columns='Type', values='Points_y')

In [ ]:
#If you want to say, make one-day races worth half, make a weighting dictionary to map values
pt_mod = {'Stage Race':1, 'One Day':0.5, 'Championships':1, 'Grand Tour':1}
pmod['Pt Adj'] = np.nan
pmod['Pt Adj'] = pmod['Type'].map(pt_mod)
pmod['One Day adj'] = pmod.Points_y * pmod['Pt Adj']
rda = pmod.groupby(['Team','Type'])['One Day adj'].sum().reset_index()
rda.pivot(index='Team', columns='Type', values='One Day adj')